In [1]:
import csv
import json
import re
# Importing reduce for
# rolling computations
from functools import reduce

import ahocorasick
import matplotlib.pyplot as plt
import pandas as pd
import requests
import requests_cache
import spacy

# Gene Extraction using Aho-Corasick

This notebook demonstrates extracting gene mentions from figure OCR text using the Aho-Corasick algorithm. This algorithm allows us to load a lexicon and find all lexicon matches in a given string. The benefit of the algorithm is that the time required to find matches doesn't depend on the number of terms in the lexicon. As long as our system has enough memory, we can load any number of lexicon terms and then find matches in constant time. This is as opposed to the implementation in `pp_classic.ipynb`, where the time required to find matches increases in proportion to the number of lexicon terms.

Much of this notebook could be easily adapted to extracting information other than gene symbols from figure OCR text, for example, chemicals and diseases. If you can create a lexicon containing terms to search for and standard IDs for each term, you should be able to update this notebook to extract those lexicon terms from the OCR text. For example, you could generate Wikidata queries or download vocabularies like MESH to serve as lexicons.

Note: the code in this notebook is only a proof of concept to demonstrate potential improvements for extraction of information from the OCR text. It is not production ready.

## Aho-Corasick Algorithm: Minimal Demo

Here's a minimal demo of the use of a Python implementation of the Aho-Corasick algorithm. We load gene symbols into an "automaton" and then find them in the sample OCR text.

In [2]:
demo_automaton = ahocorasick.Automaton()
for gene_symbol in ["A1BG", "ACE2", "AKT1"]:
    demo_automaton.add_word(
        gene_symbol,
        (
            gene_symbol,
            "symbol",
            gene_symbol,
        ),
    )
demo_automaton.make_automaton()

for (
    end_index,
    (original_word, transforms, transformed_word),
) in demo_automaton.iter("AKT1 and ACE2"):
    print(end_index, (original_word, transforms, transformed_word))

3 ('AKT1', 'symbol', 'AKT1')
12 ('ACE2', 'symbol', 'ACE2')


## Get Source Data

In [3]:
requests_cache.install_cache("pfocr_cache")

### HGNC Symbols

This will serve as our lexicon of gene names. It is intended to be equivalent to our existing gene lexicon, just updated to use the latest HGNC data. But double check it before using in production.

In [4]:
hgnc_data_url = "https://www.genenames.org/cgi-bin/download/custom?col=gd_hgnc_id&col=gd_app_sym&col=gd_app_name&col=gd_status&col=gd_prev_sym&col=gd_aliases&col=gd_pub_acc_ids&col=gd_locus_type&col=gd_date_mod&col=family.id&col=gd_locus_group&col=gd_name_aliases&col=gd_date_sym_change&col=gd_pub_eg_id&col=family.name&col=gd_date_name_change&col=gd_prev_name&col=gd_date2app_or_res&status=Approved&hgnc_dbtag=on&order_by=gd_app_sym_sort&format=text&submit=submit"
hgnc_data = []
r = requests.get(hgnc_data_url, stream=True)
if r.status_code != 200:
    raise Exception(r.text)

lines = (line.decode("utf-8") for line in r.iter_lines())
for record in csv.DictReader(lines, delimiter="\t"):
    hgnc_data.append(record)

In [5]:
len(hgnc_data)

43164

In [6]:
hgnc_data[10]

{'HGNC ID': 'HGNC:18149',
 'Approved symbol': 'A4GALT',
 'Approved name': 'alpha 1,4-galactosyltransferase (P blood group)',
 'Status': 'Approved',
 'Previous symbols': 'P1',
 'Alias symbols': 'A14GALT, Gb3S, P(k)',
 'Accession numbers': '',
 'Locus type': 'gene with protein product',
 'Date modified': '2021-04-13',
 'Gene group ID': '442|454',
 'Locus group': 'protein-coding gene',
 'Alias names': 'Gb3 synthase, "CD77 synthase", "globotriaosylceramide synthase", "lactosylceramide 4-alpha-galactosyltransferase"',
 'Date symbol changed': '',
 'NCBI Gene ID': '53947',
 'Gene group name': 'Alpha 1,4-glycosyltransferases|Blood group antigens',
 'Date name changed': '2017-03-16',
 'Previous name': 'alpha 1,4-galactosyltransferase (globotriaosylceramide synthase, P blood group), "P one antigen (P blood group)"',
 'Date approved': '2002-02-06'}

The protein coding genes:

In [7]:
protein_hgnc_data = []
for record in hgnc_data:
    if record["Locus group"] == "protein-coding gene":
        protein_hgnc_data.append(record)
len(protein_hgnc_data)

19230

In [8]:
symbol_data = []


def populate_symbol_data(
    symbol_data, symbol, status, approved_symbol, hgnc_id
):
    symbol_data.append({
        "symbol": symbol,
        "status": status,
        "approved_symbol": approved_symbol,
        "hgnc_id": hgnc_id,
    })

for record in protein_hgnc_data:
    approved_symbol = record["Approved symbol"]
    hgnc_id = record["HGNC ID"]
    populate_symbol_data(
        symbol_data, approved_symbol, "Approved", approved_symbol, hgnc_id
    )
    for symbol in record["Previous symbols"].split(", "):
        populate_symbol_data(
            symbol_data, symbol, "Previous", approved_symbol, hgnc_id
        )
    for symbol in record["Alias symbols"].split(", "):
        populate_symbol_data(
            symbol_data, symbol, "Alias", approved_symbol, hgnc_id
        )

symbol_df = pd.DataFrame.from_records(symbol_data)
symbol_df

,symbol,status,approved_symbol,hgnc_id
0,A1BG,Approved,A1BG,HGNC:5
1,,Previous,A1BG,HGNC:5
2,,Alias,A1BG,HGNC:5
3,A1CF,Approved,A1CF,HGNC:24086
4,,Previous,A1CF,HGNC:24086
...,...,...,...,...
78798,FLJ10821,Alias,ZZEF1,HGNC:29027
78799,ZZZ3,Approved,ZZZ3,HGNC:24523
78800,,Previous,ZZZ3,HGNC:24523
78801,DKFZP564I052,Alias,ZZZ3,HGNC:24523


### Figure OCR Data

Before running this, create a local symlink in the `notebooks` directory that points to the `pfocr_pipeline` Dropbox directory on your local machine.

In [9]:
import json
from pathlib import Path


for f in Path("./pfocr_pipeline/20210515/gcv_ocr").glob("*.json"):
    print(f)
    with f.open() as fp:
        ocr_output = json.load(fp)
        print(len(ocr_output))
        print(ocr_output[0])
        print(ocr_output[1])
    break

pfocr_pipeline/20210515/gcv_ocr/PMC7462072__1131f05.json
108
{'locale': 'en', 'description': "HO.\nHO.\nCOA\nNH,\nPAL\nC4H\n4CL\nCCR\nCAD\nOH\np-Coumaric acid\nOH\nOH\npCoumaraldetyde\nOH\nPhenylalanine\nCinnamic acid\npCoumaroyl CoA\np-Coumaryi alcohol\nCH4/CH3\nHCT\nно.\nOH\nShikimate\nCOMT\nCSE\nH-unit\nHO\nOCH\nOH\nOH\nOH\np-Coumaroyl shikimate\nFerulic acid\nCaffeic acid\n4CL\n4CL\nC3'H\nCOA\nCA\nShikimate\nCCOAMOT\n5-5\nOCH\nOH\nOH\nMeo\nOH OMe\nOH\nOH\nно-\nFeruloyl-CoA\nCaffeoy-CoA\nCaffeoyl shikimate\n4-0-5\na-0-4\nCR\nOMe\nMeo\nOH\nG-unit\nСAD\nно\nOMe\nOH\nMeo\nOCH3\nCHO\nOH\nOH\nOMe\nConiferaldehyde\nConiferyl alcohol\nMeo\nOH\nF5H\nFSH\nно-\nHO-\nCOMT\nCAD\nOMe\nно\nOCH,\nOCH, H,CO\nOCH\nS-unit\nLignin\nOH\nOH\nOH\n5-Hydrowy\nconiferaldetyde\nSinapaldehyde\nSinapyl alcohol\n", 'boundingPoly': {'vertices': [{'x': 7, 'y': 11}, {'x': 666, 'y': 11}, {'x': 666, 'y': 594}, {'x': 7, 'y': 594}]}}
{'description': 'HO.', 'boundingPoly': {'vertices': [{'x': 96, 'y': 11}, {'x': 109, '

## Extract Genes

Note that because we get the exact figure OCR text that was matched, we can find the position information for that text within the figure.

### Handle OCR Errors

The OCR process is good but not perfect. The OCR text sometimes has errors like having the number `0` when it should be the letter `O`. In `./pp_classic.ipynb`, we handled OCR errors by trying to transform the OCR text to "correct it" to match the HGNC gene symbols.

In this notebook, we'll take a multi-step approach. First we'll heavily normalize both the gene symbols and the OCR text to find *potential* matches.

**ocr_text** `--normalize-->` **match_text** `<--normalize--` **gene_term**

For confusable (homoglyph) characters like `0` vs. `O`, we'll convert every character from each set into one selected character from that set, so `0` would match `O`. We will also apply `casefold` to ignore differences in lower vs. uppercase. These steps are normalization, and we'll apply it to both the figure OCR text and the HGNC gene symbols so that they will match regardless of confusables or letter casing.

This normalization does cause a loss in information, but it makes it possible to efficiently find *potential* matches, cutting down the search space for a given figure from all HGNC symbols (~20k) to just the potential matches (generally up to a few hundred). Once we have a list of potential matches in a figure, we can go through each one to find the set of best non-overlapping matches.

### Non-Alphanumeric Characters

Some gene symbols include non-alphanumeric characters. Let's take a look at what characters show up for the protein-coding genes.

In [10]:
# '\W' matches underscore '_', but I actually want everything other than English letters and numbers
symbol_df["non_abc123"] = symbol_df["symbol"].str.findall("[^a-zA-Z0-9]")

In [11]:
def list_union(series):
    return reduce(lambda x, y: set(x) | set(y), series)


non_abc123_chars = symbol_df["non_abc123"].agg(list_union)
print(non_abc123_chars)

{'_', '/', '.', ',', '+', 'β', ':', 'Ã', ')', "'", 'α', '#', 'γ', '*', 'σ', '@', '(', '-'}


In [12]:
for non_abc123_char in non_abc123_chars:
    non_abc123_char_df = symbol_df[
        symbol_df["symbol"].str.contains(non_abc123_char, regex=False)
    ]
    non_abc123_char_df_len = len(non_abc123_char_df)
    print(f'"{non_abc123_char}" is in {non_abc123_char_df_len} gene symbols')
    display(non_abc123_char_df.sample(min(2, non_abc123_char_df_len), random_state=0))

"_" is in 40 gene symbols


,symbol,status,approved_symbol,hgnc_id,non_abc123
37038,H_NH0792N18.3,Alias,LUC7L2,HGNC:21608,"[_, .]"
36319,KPG_010,Alias,LPAR5,HGNC:13307,[_]


"/" is in 127 gene symbols


,symbol,status,approved_symbol,hgnc_id,non_abc123
11390,C/EBP-delta,Alias,CEBPD,HGNC:1835,"[/, -]"
27996,H2B/r,Alias,H2BC11,HGNC:4761,[/]


"." is in 1336 gene symbols


...

,symbol,status,approved_symbol,hgnc_id,non_abc123
63237,SMA@,Previous,SMN1,HGNC:11117,[@]
48364,PCDHB@,Approved,PCDHB@,HGNC:8679,[@]


"(" is in 37 gene symbols


,symbol,status,approved_symbol,hgnc_id,non_abc123
36290,Lp(a),Alias,LPA,HGNC:6667,"[(, )]"
31418,FIL1(ZETA),Alias,IL37,HGNC:15563,"[(, )]"


"-" is in 4580 gene symbols


,symbol,status,approved_symbol,hgnc_id,non_abc123
38711,CGI-63,Alias,MECR,HGNC:19691,[-]
24266,RP11-127F21,Alias,FSD2,HGNC:18024,[-]


### Gene Symbols like `AKT1` and `A26C1-3`

Many gene symbols are made up of a `base` + a numeric `suffix`, e.g., [`AKT1`](https://www.ncbi.nlm.nih.gov/gene/207) has base `AKT` + numeric suffix `1`.

Some of these gene symbols are grouped when being mentioned in pathway figures, e.g., `A26C1-3` to indicate `A26C1` and `A26C2` and `A26C3`. There are multiple ways these types of combinations are mentioned in pathway figures, and it would be computationally challenging to add all possible combinations to the lexicon. Instead, we can look for gene symbol bases like `A26C` as *potential* matches that we can follow up on by checking for whether they are followed in some way by a digit like `1` or a sequence of digits like `1-3`.

When we look for the subsequent digit(s), we can restrict our search to digits that actually exist for the base we found:

In [13]:
base_suffix_df = symbol_df[
    ["symbol", "status", "approved_symbol", "hgnc_id"]
].join(
    symbol_df["symbol"].str.extract(
        r"^(?P<base>[a-zA-Z]+[a-zA-Z0-9]*?[a-zA-Z])(?P<suffix>\d+)$"
    )
)
base_suffix_df = base_suffix_df[
    base_suffix_df["base"].notnull() &
    base_suffix_df["suffix"].notnull()
].sort_values(["base", "suffix"])
base_suffix_df

,symbol,status,approved_symbol,hgnc_id,base,suffix
51503,A26A1,Previous,POTEA,HGNC:33893,A26A,1
51508,A26B1,Previous,POTEB,HGNC:33734,A26B,1
51519,A26B2,Previous,POTEC,HGNC:33894,A26B,2
51526,A26B3,Previous,POTED,HGNC:23822,A26B,3
51535,A26C1,Alias,POTEE,HGNC:33895,A26C,1
...,...,...,...,...,...,...
57810,uS12,Alias,RPS23,HGNC:10410,uS,12
57842,uS14,Alias,RPS29,HGNC:10419,uS,14
57966,vig1,Alias,RSAD2,HGNC:30908,vig,1
31295,zcyto18,Alias,IL22,HGNC:14900,zcyto,18


In [14]:
def series_to_string(series):
    return reduce(lambda x, y: ", ".join([x, y]), series)

base_suffix_df[["base", "suffix"]].sort_values(
    ["base", "suffix"]
).groupby("base").agg(series_to_string)

,suffix
base,
A26A,1
A26B,"1, 2, 3"
A26C,"1, 2, 3"
A2BP,1
A2LD,1
...,...
uL,"18, 5"
uS,"10, 12, 14"
vig,1


In [15]:
from collections import defaultdict


base_to_suffixes = defaultdict(set)
for base, df in base_suffix_df[["base", "suffix"]].sort_values(
    ["base", "suffix"]
).groupby("base"):
    base_to_suffixes[base].update(set(df["suffix"].tolist()))

gene_symbol_bases = base_suffix_df["base"].tolist()

### Alternative Idea: Add All Variations to Automaton?

In `./pp_classic.ipynb`, we modified the OCR text to make it match the HGNC gene symbols. Could we do the opposite: modify the HGNC gene symbols so they match the OCR text? Instead of modifying `AKT| \nA26C1-3` to produce `AKT1`, `A26C1`, `A26C2` and `A26C3`, we could instead add the terms `AKT|` and `A26C1-3` to the lexicon to be matched, while maintaing a mapping from the modified lexicon terms and their originals. Since we're using the Aho-Corasick algorithm, it won't require any additional time once we've loaded the lexicon.

**ocr_text** `--normalize-->` **match_text** `<--normalize--` `<--transform--` **gene_term**

In the past, I've experimented with this, loading the variations of the HGNC gene symbols resulting from applying different permutations of normalization and transformation ([earlier version of this notebook](https://github.com/wikipathways/pathway-figure-ocr/blob/69bc0679a78061155e6063cb55d058d586822e8d/notebooks/pp_ahocorasick.ipynb)). The goal was that instead of trying to correct and parse the OCR text, we could just add every possible variation and combination of HGNC symbols to the lexicon. This way the lexicon would have a version that matches whatever is in the OCR text.

I decided against this, because the number of permutations would appear to get unmanageable. We get **40 variations** when applying a single set of confusable characters (`Il1|`) for just one gene `AKIRINI`.

In [16]:
# Get all possible order-preserving partitions of iterable
# modified from https://stackoverflow.com/a/4904492/5354298
# an alternative:
# https://more-itertools.readthedocs.io/en/stable/api.html#more_itertools.partitions
def permutations(chunks):
    for i in range(1, len(chunks)):
        start = chunks[0:i]
        end = chunks[i:]
        yield (start, end)
        for split in permutations(end):
            result = [start]
            result.extend(split)
            yield result

In [17]:
sample_confusables = "Il1|" 
ps = list(permutations(re.compile(f'[{sample_confusables}]').split("AKIRINI")))
variations = set()
for p in ps:
    chunks = p
    for char1 in list(sample_confusables):
        for char2 in list(sample_confusables):
            variation = char1.join([char2.join(chunk) for chunk in chunks])
            if variation not in variations:
                variations.add(variation)
print(len(variations))
print(", ".join(list(variations)))

40
AK|RINI, AKlRINI, AKlR1N1, AKIRINI, AKIRINl, AK|R1N|, AKIRlNl, AK1RINI, AKlRlN|, AKlRlN1, AK|R|N1, AK1R1NI, AKlR|N|, AK1R1N1, AKlR|Nl, AK1R1Nl, AKIR1N1, AK1R1N|, AK|R|NI, AK|R|N|, AK|R|Nl, AK1R|N1, AKIRlNI, AKIR1NI, AKlR1Nl, AK1RIN1, AKlRlNl, AK1R|N|, AK|RlNl, AKlRINl, AK1RlN1, AKlRlNI, AK|RIN|, AK|RlN|, AKIRIN1, AKIR|NI, AKIR|N|, AK|R1N1, AK1RlNl, AKIRIN|


 Now imagine trying to load every variation of gene symbols like `MAPK1,2`, `MAPK-1,2`, `MAPK 1,2`, `MAPK1 & 2`, `MAPK1 or 2`, `MAPK1-3`, all the way through `MAPK15`. For now, I'm just loading the normalized versions of the text. 

## Extract Genes

In [18]:
# TODO: use all of the swaps from confusables and our old transforms
# this is just a sample

# one character only
single_char_swap_sets = [
    set(["I", "l", "|", "1"]),
    set(["o", "O", "0"])
]

# at least one is longer than one character
multi_char_swap_sets = [
    set(["α", "alpha", "Alpha", "ALPHA"]),
    set(["β", "beta", "Beta", "BETA"]),
]

In [19]:
# Default is space.
PADDING_CHAR = "❌"
# TODO: could we use something like JS source maps to indicate the mappings
# from original source text characters to transformed text characters?
# We would need to update a mapping every time we did an operation like
# 'replace' so that we could indicate that the character at index 3 in
# the transformed string maps to the characters from index 3 to 6 in the
# original string, such as if we transformed '12-beta' to '12-β'.

# e.g., α -> α❌❌❌❌ a to match length of alpha
def normalize_padding(text):
    normalized = text
    for swap_set in multi_char_swap_sets:
        swap_list = sorted(swap_set, key=len)
        longest_swap_len = len(swap_list[-1])
        for swap in swap_list:
            if swap in normalized:
                # the ljust is to keep the character spacing normalized
                # so we can map from transformed to original.
                normalized = normalized.replace(
                    swap,
                    swap.ljust(longest_swap_len, PADDING_CHAR)
                )
    return normalized

# e.g., I and l both get converted to I
def merge_swaps(swap_set, text):
    # Find the shortest swap. This only matters when
    # some are multiple characters, e.g., 'α' and 'alpha'.
    swap_list = sorted(swap_set, key=len)
    shortest_swap = swap_list[0]
    longest_swap_len = len(swap_list[-1])

    normalized = text
    for swap in swap_list:
        if swap in normalized:
            # the ljust is to keep the character spacing normalized
            # so we can map from transformed to original.
            normalized = normalized.replace(
                swap,
                shortest_swap.ljust(longest_swap_len, PADDING_CHAR)
            )
    return normalized

In [20]:
def normalize_text(text):
    normalized_texts = set()

    # single-char swaps
    normalized_text = normalize_padding(text)
    for swap_set in single_char_swap_sets:
        normalized_text = merge_swaps(
            swap_set,
            normalized_text
        )
    normalized_texts.add(
        normalized_text.casefold()
    )

    # multi-char swaps
    normalized_text = text
    for swap_set in multi_char_swap_sets:
        normalized_text = merge_swaps(
            swap_set,
            normalized_text
        )
    # adding it here before doing single-char swaps
    # b/c I'm not sure the single-char swaps won't
    # mess up some multi-char swaps
    normalized_texts.add(
        normalized_text.casefold()
    )
    for swap_set in single_char_swap_sets:
        normalized_text = merge_swaps(
            swap_set,
            normalized_text
        )
    # multi- and single-char swaps
    normalized_texts.add(
        normalized_text.casefold(),
    )

    return normalized_texts

Here's what the different variations of normalized text look like:

In [21]:
normalize_text("AKlR|N1,2\nAKTl\nACE2 & ATP6AP2\nABCB5beta")

{'aklr|n1,2\naktl\nace2 & atp6ap2\nabcb5β❌❌❌',
 'ak|r|n|,2\nakt|\nace2 & atp6ap2\nabcb5beta',
 'ak|r|n|,2\nakt|\nace2 & atp6ap2\nabcb5β❌❌❌'}

In [22]:
minimum_symbol_len = 3
gene_automaton = ahocorasick.Automaton()
genes_in_automaton = set()

for gene_term in gene_symbol_bases:
    if len(gene_term) < minimum_symbol_len:
        continue

    for normalized_gene_term in normalize_text(
        gene_term
    ):
        if normalized_gene_term in genes_in_automaton:
            continue

        genes_in_automaton.add(normalized_gene_term)

        gene_automaton.add_word(
            normalized_gene_term,
            (
                gene_term,
                "base",
                normalized_gene_term,
            ),
        )

for gene_term in symbol_df["symbol"].tolist():
    if len(gene_term) < minimum_symbol_len:
        continue

    for normalized_gene_term in normalize_text(
        gene_term
    ):
        if normalized_gene_term in genes_in_automaton:
            continue

        genes_in_automaton.add(normalized_gene_term)

        gene_automaton.add_word(
            normalized_gene_term,
            (
                gene_term,
                "symbol",
                normalized_gene_term,
            ),
        )

gene_automaton.make_automaton()

In [23]:
sample_ocr_text = "AKIRlN1,2\n"
for normalized_text in normalize_text(sample_ocr_text):
    for (
        end_index,
        (original_word, transforms, transformed_word),
    ) in gene_automaton.iter(normalized_text):
        print((end_index, original_word, transforms, transformed_word))

(3, 'KIR', 'base', 'kir')
(5, 'RLN', 'base', 'rln')
(6, 'RLN1', 'symbol', 'rln1')
(2, 'AK1', 'symbol', 'ak|')
(3, 'KIR', 'base', 'k|r')
(5, 'AKIRIN', 'base', 'ak|r|n')
(5, 'RIN', 'base', 'r|n')
(6, 'AKIRIN1', 'symbol', 'ak|r|n|')
(6, 'RIN1', 'symbol', 'r|n|')
(6, 'INI', 'symbol', '|n|')


Here are some examples in DataFrames with surrounding context for figure OCR text:

In [24]:
def generate_match_df(text, remove_padding=False):
    context_len = 8
    padding_normalized_text = normalize_padding(text)
    match_data = []
    for normalized_text in normalize_text(text):
        for (
            end_index,
            (lexicon_term, transforms, normalized_text),
        ) in gene_automaton.iter(normalized_text):
            start_index = end_index - len(normalized_text) + 1

            pre_context = padding_normalized_text[
                max(0, end_index - len(normalized_text) - context_len):
                start_index
            ]
            matched_ocr_text = padding_normalized_text[
                start_index:
                end_index + 1
            ]
            post_context = padding_normalized_text[
                end_index + 1:
                end_index + context_len
            ]
            match_data.append({
                # NOTE: these indices include any padding characters
                # added to the OCR text
                #"index_interval": pd.Interval(start_index, end_index, closed="both"),
                "start_index": start_index,
                "end_index": end_index,
                "pre_context": pre_context,
                "ocr_text": matched_ocr_text,
                "post_context": post_context,
                "normalized_text": normalized_text,
                "lexicon_term": lexicon_term,
                "lexicon_term_type": transforms,
            })
    match_df = pd.DataFrame.from_records(match_data).sort_values(["start_index", "end_index"])
    if remove_padding:
        match_df["pre_context"] = match_df["pre_context"].str.replace(PADDING_CHAR, "")
        match_df["ocr_text"] = match_df["ocr_text"].str.replace(PADDING_CHAR, "")
        match_df["post_context"] = match_df["post_context"].str.replace(PADDING_CHAR, "")
        match_df["normalized_text"] = match_df["normalized_text"].str.replace(
            PADDING_CHAR, ""
        )
        match_df["lexicon_term"] = match_df["lexicon_term"].str.replace(PADDING_CHAR, "")
    match_df["overlap_group"] = (match_df["start_index"] > match_df["end_index"].shift()).cumsum()
    return match_df

In [25]:
sample_text = "\nABcB5beta\n AKT|-3"
match_df = generate_match_df(sample_text)
match_df

,start_index,end_index,pre_context,ocr_text,post_context,normalized_text,lexicon_term,lexicon_term_type,overlap_group
0,1,3,\n,ABc,B5beta\n,abc,ABC,base,0
9,1,3,\n,ABc,B5beta\n,abc,ABC,base,0
1,1,4,\n,ABcB,5beta\n,abcb,ABCB,base,0
10,1,4,\n,ABcB,5beta\n,abcb,ABCB,base,0
2,1,5,\n,ABcB5,beta\n A,abcb5,ABCB5,symbol,0
11,1,5,\n,ABcB5,beta\n A,abcb5,ABCB5,symbol,0
4,1,9,\n,ABcB5beta,\n AKT|-,abcb5beta,ABCB5beta,symbol,0
12,1,9,\n,ABcB5beta,\n AKT|-,abcb5β❌❌❌,ABCB5beta,symbol,0
3,6,8,\nABcB5,bet,a\n AKT|,bet,BET,base,0
5,6,9,\nABcB5,beta,\n AKT|-,beta,BETA,base,0


In [26]:
sample_text = "\nABcB5β\n AKT|-3"
match_df = generate_match_df(sample_text)
match_df

,start_index,end_index,pre_context,ocr_text,post_context,normalized_text,lexicon_term,lexicon_term_type,overlap_group
0,1,3,\n,ABc,B5β❌❌❌\n,abc,ABC,base,0
1,1,4,\n,ABcB,5β❌❌❌\n,abcb,ABCB,base,0
2,1,5,\n,ABcB5,β❌❌❌\n A,abcb5,ABCB5,symbol,0
3,1,9,\n,ABcB5β❌❌❌,\n AKT|-,abcb5β❌❌❌,ABCB5beta,symbol,0
4,6,9,\nABcB5,β❌❌❌,\n AKT|-,β❌❌❌,BETA,base,0
5,12,14,cB5β❌❌❌\n,AKT,|-3,akt,AKT,base,1
6,12,15,cB5β❌❌❌\n,AKT|,-3,akt|,AKT1,symbol,1
7,13,15,B5β❌❌❌\n A,KT|,-3,kt|,KTI,base,1


Now that we have potential matches for gene symbols and/or gene symbol bases, we can analyze which ones to pursue further. In the case of overlapping potential matches, the following factors can help determine which one to choose:

1. Length of match: longer matches are more reliable.
2. Levenshtein distance between gene symbol and OCR text. But take into account that a mismatch of a confusable homoglyph isn't the same as other mismatches. `AKT|` is more likely to match `AKT1` than `AKT3`.
3. Use gene symbol bases and their associated numeric suffixes to investigate whether a match is a case like `A26B1-3` -> `{A26B1,A26B2,A26B3}` or `KRTAP1-3`, which is a full gene symbol on its own.

The code below is an initial rough attempt to demo these ideas, but it needs further work.

In [48]:
import re


combo_suffix_pattern = re.compile(
    r"[\ \t]?(?P<start_digit>\d+)(?P<connector>([\-,\&\|]|(\ ?and\ ?)|(\ ?or\ ?)))(?P<end_digit>\d+)"
)

def extract_lexicon_terms(text):
    match_df = generate_match_df(text, remove_padding=True)
    extracted_data = []
    for overlap_group_index, overlap_group_df in match_df.groupby("overlap_group", sort=False):
        longest_extracted = ""
        extracted_data_this_overlap_group = []

        overlap_group_df["ocr_text_len"] = overlap_group_df["ocr_text"].map(len)

        longest_extracted_row = overlap_group_df.sort_values(
            "ocr_text_len", ascending=False
        ).iloc[0]
        longest_extracted_from_symbols = longest_extracted_row["ocr_text"]
        lexicon_term = longest_extracted_row["lexicon_term"]

        symbol_match_df = symbol_df[symbol_df["symbol"] == lexicon_term]
        if len(symbol_match_df) > 0:
            source_data_row = symbol_match_df.iloc[0]
            extracted_data_this_overlap_group.append({
                "ocr_text": longest_extracted_from_symbols,
                # "start_index": longest_extracted_row["start_index"],
                # "end_index": longest_extracted_row["end_index"],
                "matched_term": longest_extracted_from_symbols,
                "approved_symbol": source_data_row["approved_symbol"],
                "hgnc_id": source_data_row["hgnc_id"],
            })
            longest_extracted = longest_extracted_from_symbols

        for i, df in overlap_group_df[overlap_group_df["lexicon_term_type"] == "base"].iterrows():
            ocr_text = df["ocr_text"]
            post_context = df["post_context"]
            lexicon_term = df["lexicon_term"]
            m = combo_suffix_pattern.match(post_context)
            if m:
                combo_suffix = m.group(0)
                suffixes = base_to_suffixes[lexicon_term]
                start_digit = m.group("start_digit")
                connector = m.group("connector")
                end_digit = m.group("end_digit")

                while len(end_digit) > 0:
                    if end_digit in suffixes:
                        break
                    else:
                        end_digit = end_digit[:-1]
                if end_digit not in suffixes:
                        continue

                extracted_from_base = lexicon_term + start_digit + connector + end_digit
                if len(extracted_from_base) > len(longest_extracted):
                    longest_extracted = extracted_from_base
                    extracted_data_this_overlap_group = []

                for digit in range(int(start_digit), int(end_digit) + 1):
                    matched_term = ocr_text + str(digit)
                    source_data_row = symbol_df[symbol_df["symbol"] == matched_term].iloc[0]
                    extracted_data_this_overlap_group.append({
                        "ocr_text": extracted_from_base,
                        # "start_index": df["start_index"],
                        # "end_index": df["end_index"] + len(start_digit + connector + end_digit),
                        "matched_term": matched_term,
                        "approved_symbol": source_data_row["approved_symbol"],
                        "hgnc_id": source_data_row["hgnc_id"],
                    })

        longest_extracted = longest_extracted_from_symbols
        extracted_data += extracted_data_this_overlap_group

        # print(f"longest extracted from OCR text in overlap group {overlap_group_index}: {longest_extracted}")
    return extracted_data

In [49]:
extract_lexicon_terms("AKT1-312R-LOX")

[{'ocr_text': 'AKT1-3',
  'matched_term': 'AKT1',
  'approved_symbol': 'AKT1',
  'hgnc_id': 'HGNC:391'},
 {'ocr_text': 'AKT1-3',
  'matched_term': 'AKT2',
  'approved_symbol': 'AKT2',
  'hgnc_id': 'HGNC:392'},
 {'ocr_text': 'AKT1-3',
  'matched_term': 'AKT3',
  'approved_symbol': 'AKT3',
  'hgnc_id': 'HGNC:393'},
 {'ocr_text': 'AKT1-3',
  'matched_term': 'AKT1',
  'approved_symbol': 'AKT1',
  'hgnc_id': 'HGNC:391'},
 {'ocr_text': 'AKT1-3',
  'matched_term': 'AKT2',
  'approved_symbol': 'AKT2',
  'hgnc_id': 'HGNC:392'},
 {'ocr_text': 'AKT1-3',
  'matched_term': 'AKT3',
  'approved_symbol': 'AKT3',
  'hgnc_id': 'HGNC:393'},
 {'ocr_text': '12R-LOX',
  'matched_term': '12R-LOX',
  'approved_symbol': 'ALOX12B',
  'hgnc_id': 'HGNC:430'}]

The code above works for some inputs, but it fails for other inputs like the following:

In [50]:
sample_text = "WASPARCAND1"
extract_lexicon_terms(sample_text)

[{'ocr_text': 'WASPA',
  'matched_term': 'WASPA',
  'approved_symbol': 'WAS',
  'hgnc_id': 'HGNC:12731'}]

It probably should extract `WASPA` and `CAND1`, but it fails because every character is part of multiple mutually overlapping gene symbol matches, so there is only one overlap group.

In [51]:
sample_text = "WASPARCAND1"
match_df = generate_match_df(sample_text, remove_padding=True)
match_df

,start_index,end_index,pre_context,ocr_text,post_context,normalized_text,lexicon_term,lexicon_term_type,overlap_group
0,0,2,,WAS,PARCAND,was,WAS,symbol,0
18,0,2,,WAS,PARCAND,was,WAS,symbol,0
1,0,3,,WASP,ARCAND1,wasp,WASP,symbol,0
19,0,3,,WASP,ARCAND1,wasp,WASP,symbol,0
3,0,4,,WASPA,RCAND1,waspa,WASPA,symbol,0
21,0,4,,WASPA,RCAND1,waspa,WASPA,symbol,0
2,1,3,W,ASP,ARCAND1,asp,ASP,symbol,0
20,1,3,W,ASP,ARCAND1,asp,ASP,symbol,0
4,1,4,W,ASPA,RCAND1,aspa,ASPA,symbol,0
22,1,4,W,ASPA,RCAND1,aspa,ASPA,symbol,0


In [52]:
import itertools
import Levenshtein
import sys


extracted = []
for f in itertools.islice(
    Path("./pfocr_pipeline/20210515/gcv_ocr").glob("*.json"),
    10
):
    with f.open() as fp:
        ocr_output = json.load(fp)
    full_text = ocr_output[0]["description"]
    for extracted_item in extract_lexicon_terms(full_text):
        extracted_ocr_text = extracted_item["ocr_text"]
        smallest_distance = sys.maxsize
        for ocr_entry in ocr_output[1:]:
            ocr_block_text = ocr_entry["description"]
            ocr_block_position = ocr_entry["boundingPoly"]
            # TODO: may need to combine blocks to get the match
            distance = Levenshtein.distance(extracted_ocr_text, ocr_block_text)
            if distance < smallest_distance:
                smallest_distance = distance
                extracted_item["position"] = ocr_block_position
                extracted_item["text_at_position"] = ocr_block_text
        extracted_item["figure_id"] = f.stem + ".jpg"
        extracted.append(extracted_item)
extracted_df = pd.DataFrame.from_records(extracted)
extracted_df

,ocr_text,matched_term,approved_symbol,hgnc_id,position,text_at_position,figure_id
0,PAL,PAL,LRIT1,HGNC:23404,"{'vertices': [{'x': 59, 'y': 47}, {'x': 72, 'y...",PAL,PMC7462072__1131f05.jpg
1,CAD,CAD,ACOD1,HGNC:33904,"{'vertices': [{'x': 377, 'y': 51}, {'x': 393, ...",CAD,PMC7462072__1131f05.jpg
2,aral,aral,SART1,HGNC:10538,"{'vertices': [{'x': 200, 'y': 90}, {'x': 210, ...",acid,PMC7462072__1131f05.jpg
3,lal,lal,LIPA,HGNC:6617,"{'vertices': [{'x': 96, 'y': 11}, {'x': 109, '...",HO.,PMC7462072__1131f05.jpg
4,Cin,Cin,PDXP,HGNC:30259,"{'vertices': [{'x': 240, 'y': 14}, {'x': 258, ...",COA,PMC7462072__1131f05.jpg
...,...,...,...,...,...,...,...
230,ell,ell,CEL,HGNC:1848,"{'vertices': [{'x': 29, 'y': 185}, {'x': 59, '...",Cell,PMC7509689__ol-20-05-12112-g08.jpg
231,MPK,MPK,PRKAA2,HGNC:9377,"{'vertices': [{'x': 540, 'y': 45}, {'x': 574, ...",MPA,PMC7509689__ol-20-05-12112-g08.jpg
232,roli,roli,LMNA,HGNC:6636,"{'vertices': [{'x': 29, 'y': 185}, {'x': 59, '...",Cell,PMC7509689__ol-20-05-12112-g08.jpg
233,GR-,GR-,PGR,HGNC:8910,"{'vertices': [{'x': 543, 'y': 205}, {'x': 592,...",PGR-B,PMC7509689__ol-20-05-12112-g08.jpg


In [53]:
extracted_df[extracted_df["ocr_text"].str.contains("\d+\-\d+")]

,ocr_text,matched_term,approved_symbol,hgnc_id,position,text_at_position,figure_id
166,14-3-3,14-3-3,YWHAQ,HGNC:12854,"{'vertices': [{'x': 566, 'y': 197}, {'x': 599,...",14-3-3,PMC6557283__nihms-1525527-f0001.jpg
186,TEAD1-4,TEAD1,TEAD1,HGNC:11714,"{'vertices': [{'x': 393, 'y': 415}, {'x': 443,...","TEAD1-4,",PMC6557283__nihms-1525527-f0001.jpg
187,TEAD1-4,TEAD2,TEAD2,HGNC:11715,"{'vertices': [{'x': 393, 'y': 415}, {'x': 443,...","TEAD1-4,",PMC6557283__nihms-1525527-f0001.jpg
188,TEAD1-4,TEAD3,TEAD3,HGNC:11716,"{'vertices': [{'x': 393, 'y': 415}, {'x': 443,...","TEAD1-4,",PMC6557283__nihms-1525527-f0001.jpg
189,TEAD1-4,TEAD4,TEAD4,HGNC:11717,"{'vertices': [{'x': 393, 'y': 415}, {'x': 443,...","TEAD1-4,",PMC6557283__nihms-1525527-f0001.jpg
190,TEAD1-4,TEAD1,TEAD1,HGNC:11714,"{'vertices': [{'x': 393, 'y': 415}, {'x': 443,...","TEAD1-4,",PMC6557283__nihms-1525527-f0001.jpg
191,TEAD1-4,TEAD2,TEAD2,HGNC:11715,"{'vertices': [{'x': 393, 'y': 415}, {'x': 443,...","TEAD1-4,",PMC6557283__nihms-1525527-f0001.jpg
192,TEAD1-4,TEAD3,TEAD3,HGNC:11716,"{'vertices': [{'x': 393, 'y': 415}, {'x': 443,...","TEAD1-4,",PMC6557283__nihms-1525527-f0001.jpg
193,TEAD1-4,TEAD4,TEAD4,HGNC:11717,"{'vertices': [{'x': 393, 'y': 415}, {'x': 443,...","TEAD1-4,",PMC6557283__nihms-1525527-f0001.jpg
